In [1]:
import sys
from pathlib import Path
import os
cwd = os.getcwd()
parent = str(Path(cwd).parents[0])
sys.path.append(parent)

In [96]:
import pandas as pd
import numpy as np

In [52]:
raw_extract = pd.read_csv(parent + '/data/interventions/2022-05-26_034658-form_7-refset_34-extractions.tsv', sep='\t', encoding="ISO-8859-1")
raw_arms = pd.read_csv(parent + '/data/interventions/2022-05-26_034703-sub_arm-refset_34-sub-extraction.tsv', sep='\t', encoding="ISO-8859-1")
rec_extract = pd.read_csv(parent + '/data/interventions/2022-05-26_034706-form_7-refset_34-final.tsv', sep='\t', encoding="ISO-8859-1")
rec_arms = pd.read_csv(parent + '/data/interventions/2022-05-26_034711-sub_arm-refset_34-sub-final.tsv', sep='\t', encoding="ISO-8859-1")

In [53]:
reconciled_ids = rec_arms.db_id.to_list()

In [54]:
remove_rec = raw_arms[~raw_arms.db_id.isin(reconciled_ids)]

In [98]:
cols = ['db_id', 'trialid', 'type', 'placebo_plus_soc', 'category', 'intervention', 'intervention_plus_soc']

In [56]:
final_int = pd.concat([remove_rec[cols], rec_arms[cols]]).reset_index(drop=True)

In [93]:
final_exp = final_int[final_int.type == 'experimental'].reset_index(drop=True)

In [97]:
final_exp['HCQ'] = np.where(final_exp.intervention.str.contains('Hydroxychloroquine'), 1, 0)
final_exp['IVE'] = np.where(final_exp.intervention.str.contains('Ivermectin'), 1, 0)
final_exp['CP'] = np.where(final_exp.intervention.str.contains('Convalescent Plasma'), 1, 0)

In [104]:
final_exp[['db_id', 'HCQ', 'IVE', 'CP']].groupby('db_id').sum().to_csv(parent + '/data/interventions/common_ints.csv')

In [88]:
ints = final_int[final_int.type == 'experimental'].intervention

In [81]:
ints

0          Traditional Medicine
2          Traditional Medicine
4          Traditional Medicine
6                      Azvudine
8          Traditional Medicine
                 ...           
3922            Povidone-iodine
3923    Chlorhexidine gluconate
3925                 Amiodarone
3926                  Verapamil
3928                 Ivermectin
Name: intervention, Length: 2470, dtype: object

In [89]:
ints_exp = final_int[final_int.type == 'experimental']

In [ ]:
ints_exp[ints_exp.

In [86]:
final_int[final_int.intervention == 'Dabur chyawanprash']

,db_id,trialid,type,placebo_plus_soc,category,intervention,intervention_plus_soc


In [59]:
int_list = ints.to_list()

In [61]:
all_ints = []
for i in int_list:
    if ';' in i:
        all_ints = all_ints + i.split(';')
    else:
        all_ints.append(i)

In [63]:
import collections
c = collections.Counter(x for x in all_ints if x)

In [76]:
int_counts = pd.DataFrame.from_dict(dict(c), orient='index').reset_index()
int_counts.columns = ['interventions', 'counts']

In [80]:
int_counts.sort_values(by='counts', ascending=False).head(10)

,interventions,counts
0,Traditional Medicine,223
32,Hydroxychloroquine,198
10,Convalescent Plasma,135
40,Ivermectin,64
23,Ritonavir,61
33,Azithromycin,58
22,Lopinavir,54
18,Tocilizumab,51
13,Favipiravir,51
28,Chloroquine,41


In [83]:
int_counts.to_csv(parent + '/data/interventions/unique_ints.csv')